In [109]:
import sys
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.cross_validation import train_test_split
sys.path.insert(0, '../../notebooks/libs/')
import FeatureExtraction as FE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [10]:
import sys
sys.path.insert(0, '../libs/')
%matplotlib inline

import FeatureExtraction
from lxml import etree

training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

sexual_predator_ids_file = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'

chat_based_features_csv_train='../../csv/chat_based_features_training.csv'
chat_based_features_csv_test='../../csv/chat_based_features_test.csv'

In [11]:
def write_statistics(y_test,predicted_test):
    print("\ntest statistics")
    print('acc',accuracy_score(y_test,predicted_test))
    print('rec',recall_score(y_test,predicted_test))
    print('prec',precision_score(y_test,predicted_test))
    print('f1',f1_score(y_test,predicted_test))
    print('f0.5',fbeta_score(y_test,predicted_test,0.5))

In [4]:
minimal_number_of_messages_treshold=5

In [5]:
documents_training=FE.prepare_for_tf_idf(training_xml,False,True,minimal_number_of_messages_treshold)
document_testing=FE.prepare_for_tf_idf(test_xml,False,True,minimal_number_of_messages_treshold)

In [7]:
train_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_train))
test_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_test))
train_frame=train_frame[train_frame['number of messages sent'] >=minimal_number_of_messages_treshold]
test_frame=test_frame[test_frame['number of messages sent']>=minimal_number_of_messages_treshold]
print(len(train_frame))
print(len(test_frame))
X_train_chat_based=train_frame.ix[:,1:-1]
y_train=np.ravel(train_frame[[-1]])
X_test_chat_based=test_frame.ix[:,1:-1]
y_test=np.ravel(test_frame[[-1]])

22063
47458


In [8]:
tfidf=TfidfVectorizer(stop_words='english',min_df=3,max_features=3500)
matrix_training=tfidf.fit_transform(documents_training)
matrix_testing=tfidf.transform(document_testing)

In [137]:
classifier=svm.SVC(C=0.7454,kernel='linear',max_iter=100)
classifier.fit(matrix_training,y_train)
predicted_test=classifier.predict(matrix_testing)
write_statistics(y_test,predicted_test)


test statistics
('acc', 0.9969868093893548)
('rec', 0.45833333333333331)
('prec', 0.89430894308943087)
('f1', 0.60606060606060608)
('f0.5', 0.75136612021857929)


In [141]:
Cs=[2**x for x in range(-5,15)]
gammas=[2**x for x in range (-15,3)]




In [142]:
linsvm= svm.SVC(kernel='linear',max_iter=100,random_state=5)
est=clf = make_pipeline(StandardScaler(with_mean=False), linsvm)

print('koraci',est.steps)
#grid_search=GridSearchCV(estimator=est,param_grid=dic_for_pipe,n_jobs=8)
grid_search=GridSearchCV(estimator=svm.SVC(),param_grid={'C':Cs,'gamma':gammas},n_jobs=8)
grid_search

('koraci', [('standardscaler', StandardScaler(copy=True, with_mean=False, with_std=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=100, probability=False, random_state=5, shrinking=True,
  tol=0.001, verbose=False))])


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=8,
       param_grid={'C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384], 'gamma': [3.0517578125e-05, 6.103515625e-05, 0.0001220703125, 0.000244140625, 0.00048828125, 0.0009765625, 0.001953125, 0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [143]:
grid_search.fit(matrix_training,y_train)
pred=grid_search.predict(matrix_testing)
write_statistics(y_test,pred)


test statistics
('acc', 0.99686038181128578)
('rec', 0.40416666666666667)
('prec', 0.94174757281553401)
('f1', 0.56559766763848385)
('f0.5', 0.74386503067484666)


In [144]:
grid_search.best_estimator_


SVC(C=8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.125, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)